# Run Test Portfolio (5 sims)

Builds a minimal CSV from `test_portfolio.xlsx` and runs `Simulate_Cashflows.ipynb` using existing calibrations.

In [76]:
from pathlib import Path
import os
import numpy as np
import pandas as pd

ROOT = Path.cwd()
if ROOT.name == "model_fits":
    ROOT = ROOT.parent

# --- Config ---
INPUT_XLSX = ROOT / "test_portfolio.xlsx"

# Auto-detect test_portfolio.xlsx if not found
if not INPUT_XLSX.exists():
    candidates = list(ROOT.glob("**/test_portfolio.xlsx"))
    if not candidates:
        candidates = list(ROOT.parent.glob("**/test_portfolio.xlsx"))
    if candidates:
        INPUT_XLSX = candidates[0]

OUTPUT_CSV = ROOT / "model_fits" / "test_portfolio.csv"
RUN_TAG = "test_portfolio_2025Q3_Sentral"

CALIB_SOURCE_TAG = "2025Q3"
CUTOFF = "2025Q3"
N_SIMS = 1000
RUN_NOTEBOOK = True


In [77]:
def quarter_end_from_qstr(qstr: str) -> pd.Timestamp:
    qstr = qstr.strip().upper()
    if "Q" not in qstr:
        raise ValueError("cutoff must be like 2025Q3")
    year = int(qstr[:4])
    q = int(qstr[-1])
    return pd.Period(f"{year}Q{q}", freq="Q").to_timestamp("Q")


def compute_avg_life_by_strategy(hist_path: Path) -> tuple[dict, float]:
    if not hist_path.exists():
        return {}, 12.0
    df = pd.read_csv(hist_path)
    if "Adj Strategy" not in df.columns and "Adj strategy" in df.columns:
        df = df.rename(columns={"Adj strategy": "Adj Strategy"})
    if "Planned End Date" not in df.columns:
        for col in [
            "Planned end date with add. years as per legal doc",
            "Planned End Date as per legal documentation",
        ]:
            if col in df.columns:
                df = df.rename(columns={col: "Planned End Date"})
                break
    if "First Closing Date" not in df.columns or "Planned End Date" not in df.columns or "Adj Strategy" not in df.columns:
        return {}, 12.0

    df["First Closing Date"] = pd.to_datetime(df["First Closing Date"], errors="coerce")
    df["Planned End Date"] = pd.to_datetime(df["Planned End Date"], errors="coerce")
    df = df.dropna(subset=["First Closing Date", "Planned End Date", "Adj Strategy"])
    if df.empty:
        return {}, 12.0

    years = (df["Planned End Date"] - df["First Closing Date"]).dt.days / 365.25
    df = df.assign(life_years=years)
    df = df[(df["life_years"] > 0) & (df["life_years"] < 50)]
    if df.empty:
        return {}, 12.0

    avg_by_strategy = df.groupby("Adj Strategy")["life_years"].mean().to_dict()
    global_avg = float(df["life_years"].mean()) if len(df) else 12.0
    if not (global_avg and global_avg > 0):
        global_avg = 12.0
    return avg_by_strategy, global_avg


def make_test_csv(xlsx_path: Path, out_csv: Path, cutoff_q: str) -> Path:
    df = pd.read_excel(xlsx_path)
    rename = {
        "VC Fund Name": "FundID",
        "Strategy": "Adj Strategy",
        "Grading": "Grade",
        "First Closing Date": "First Closing Date",
        "Commitment amount": "Commitment EUR",
        "Drawn amount": "Adj Drawdown EUR",
        "Reflows amount": "Adj Repayment EUR",
        "Adjusted NAV": "NAV Adjusted EUR",
    }
    df = df.rename(columns=rename)

    cutoff_qe = quarter_end_from_qstr(cutoff_q)
    cutoff_period = pd.Period(cutoff_qe, freq="Q")
    year = cutoff_period.year
    quarter = cutoff_period.quarter

    df["FundID"] = df["FundID"].astype(str)
    df["Adj Strategy"] = df["Adj Strategy"].astype(str)
    df["Grade"] = df["Grade"].astype(str)
    df["Fund Workflow Stage"] = "Active"
    df["Year of Transaction Date"] = year
    df["Quarter of Transaction Date"] = quarter
    df["Transaction Quarter"] = f"{year}-Q{quarter}"

    for col in ["Commitment EUR", "Adj Drawdown EUR", "Adj Repayment EUR", "NAV Adjusted EUR"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0.0)
        else:
            df[col] = 0.0

    df["draw_cum_prev"] = df["Adj Drawdown EUR"].fillna(0.0)
    df["Recallable"] = 0.0
    df["Recallable_Percentage_Decimal"] = 0.0
    df["Expiration_Quarters"] = 0

    df["First Closing Date"] = pd.to_datetime(df["First Closing Date"], errors="coerce")
    age_q = []
    for fc in df["First Closing Date"]:
        if pd.isna(fc):
            age_q.append(0)
            continue
        fc_q = pd.Period(fc, freq="Q")
        age_q.append(max(0, cutoff_period.ordinal - fc_q.ordinal))
    df["Fund_Age_Quarters"] = age_q

    if "VC Fund Status" not in df.columns:
        df["VC Fund Status"] = ""

    # Planned End Date: use avg life by strategy from anonymized.csv
    avg_map, global_avg = compute_avg_life_by_strategy(Path("anonymized.csv"))
    if "Planned End Date" not in df.columns:
        df["Planned End Date"] = pd.NaT
    for idx, row in df.iterrows():
        if pd.notna(row.get("Planned End Date")):
            continue
        fc = row.get("First Closing Date", pd.NaT)
        if pd.isna(fc):
            continue
        strat = row.get("Adj Strategy", "")
        yrs = float(avg_map.get(strat, global_avg))
        df.at[idx, "Planned End Date"] = fc + pd.DateOffset(years=int(round(yrs)))

    out_csv.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(out_csv, index=False)
    return out_csv


In [78]:

# --- Build test portfolio CSV ---

csv_path = make_test_csv(INPUT_XLSX, OUTPUT_CSV, CUTOFF)
print("Wrote test portfolio CSV:", csv_path)

if not RUN_NOTEBOOK:
    print("RUN_NOTEBOOK=False, stopping here.")
    raise SystemExit(0)

from pathlib import Path
import nbformat
from nbclient import NotebookClient

ROOT = Path.cwd()
if ROOT.name == "model_fits":
    ROOT = ROOT.parent

# --- Ensure calibration files are available for this run ---
calib_candidates = [
    ROOT / "model_fits" / "runs" / CALIB_SOURCE_TAG / "calibration",
]
calib_src = next((p for p in calib_candidates if p.exists()), None)
if calib_src is None:
    raise FileNotFoundError(f"Calibration folder not found for tag {CALIB_SOURCE_TAG}. Searched: {calib_candidates}")

calib_dst = ROOT / "model_fits" / "runs" / RUN_TAG / "calibration"
calib_dst.mkdir(parents=True, exist_ok=True)

def _csv_has_rows(p: Path) -> bool:
    try:
        import pandas as pd
        df = pd.read_csv(p)
        return len(df) > 0
    except Exception:
        return False

def _nav_targets_nonempty(p: Path) -> bool:
    try:
        import pandas as pd
        df = pd.read_csv(p)
        if "avg_nav_to_paidin" in df.columns:
            return df["avg_nav_to_paidin"].notna().any()
        return len(df) > 0
    except Exception:
        return False

copy_files = [
    'ratio_fit_selected.csv',
    'timing_probs_selected.csv',
    'omega_selected.csv',
    'copula_params.json',
]
for fname in copy_files:
    src = calib_src / fname
    if src.exists():
        (calib_dst / fname).write_bytes(src.read_bytes())

# NAV anchor files (prefer whole-portfolio calibration if present and non-empty)
nav_targets = calib_src / 'nav_anchor_targets.csv'
nav_cal = calib_src / 'nav_anchor_calibration.csv'
if nav_targets.exists() and _nav_targets_nonempty(nav_targets):
    (calib_dst / 'nav_anchor_targets.csv').write_bytes(nav_targets.read_bytes())
else:
    raise FileNotFoundError(f'nav_anchor_targets.csv missing or empty in {calib_src}')

if nav_cal.exists() and _csv_has_rows(nav_cal):
    (calib_dst / 'nav_anchor_calibration.csv').write_bytes(nav_cal.read_bytes())
else:
    raise FileNotFoundError(f'nav_anchor_calibration.csv missing or empty in {calib_src}')


# --- Run Simulation ---
sim_nb = ROOT / "model_fits" / "Simulate_Cashflows.ipynb"
if not sim_nb.exists():
    raise FileNotFoundError(f"Missing notebook: {sim_nb}")

os.environ.update({
    "INPUT_PATH": str(csv_path),
    "RUN_TAG": RUN_TAG,
    "HIST_END": CUTOFF,
    "START_FROM_HIST_END": "1",
    "N_SIMS": str(N_SIMS),
    "FIT_DIR": str(calib_dst),
    "GRADE_UPDATE_ENABLED": "0",
    "GRADE_USE_INPUT": "1",
    "SMALL_SAMPLE_RULE_ENABLED": "0",
    "NAV_ANCHOR_ENABLED": "0",  # disable NAV anchoring for test run
})

nb = nbformat.read(sim_nb, as_version=4)
client = NotebookClient(nb, timeout=None, kernel_name="python3", resources={"metadata": {"path": str(ROOT)}})
client.execute()

print("Done. Outputs in:", ROOT / "model_fits" / "runs" / RUN_TAG / "projection" / "sim_outputs")

# --- Quick post-run diagnostics ---
try:
    import pandas as pd
    out_dir = ROOT / "model_fits" / "runs" / RUN_TAG / "projection" / "sim_outputs"
    series_path = out_dir / "sim_portfolio_series.csv"
    if series_path.exists():
        df = pd.read_csv(series_path)
        rep_sum = float(df["sim_rep_mean"].sum())
        draw_sum = float(df["sim_draw_mean"].sum())
        nav_end = float(df["sim_nav_mean"].iloc[-1]) if len(df) else float("nan")
        print(f"Diagnostics: rep_sum={rep_sum:.2f}, draw_sum={draw_sum:.2f}, nav_end={nav_end:.2f}")
    repdiag_path = out_dir / "repayment_diagnostics_by_quarter.csv"
    if repdiag_path.exists():
        rd = pd.read_csv(repdiag_path)
        print("Repayment diag sums:", rd[["rep_event_count","rep_navpos_count","rep_rr_missing","rep_prep_nan","timing_key_missing"]].sum().to_dict())
    else:
        print("Repayment diagnostics file missing:", repdiag_path)
except Exception as e:
    print("Post-run diagnostics skipped:", e)



Wrote test portfolio CSV: /Users/mozeramozali/Desktop/Equity-Cashflow-projection/model_fits/test_portfolio.csv
Done. Outputs in: /Users/mozeramozali/Desktop/Equity-Cashflow-projection/model_fits/runs/test_portfolio_2025Q3_Sentral/projection/sim_outputs
Diagnostics: rep_sum=1166010758.07, draw_sum=1384240980.24, nav_end=55340531.00
Repayment diag sums: {'rep_event_count': 477275.0, 'rep_navpos_count': 1716317.0, 'rep_rr_missing': 0.0, 'rep_prep_nan': 0.0, 'timing_key_missing': 0.0}
